In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import time

from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
news_url = 'https://redplanetscience.com/'
featured_img = 'https://spaceimages-mars.com/'
table_url = 'https://galaxyfacts-mars.com/'
hemispheres_url = 'https://marshemispheres.com/'

In [3]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Downloading: 100%|██████████| 6.68M/6.68M [00:00<00:00, 17.7MB/s]


## News URL

In [4]:
browser.visit(news_url)

time.sleep(1)

In [5]:
html = browser.html
soup = bs(html, 'html.parser')


In [6]:
news_title = soup.find_all('div', class_='content_title')[0].text
news_p = soup.find_all('div', class_='article_teaser_body')[0].text

In [7]:
news_title

"NASA's Perseverance Rover Will Carry First Spacesuit Materials to Mars"

In [8]:
news_p

'In a Q&A, spacesuit designer Amy Ross explains how five samples, including a piece of helmet visor, will be tested aboard the rover, which is targeting a July 30 launch. '

In [9]:
# browser.quit()

## Featured Image

In [10]:
browser.visit(featured_img)

time.sleep(1)

In [11]:
browser.links.find_by_partial_text('FULL IMAGE').click()

In [12]:
html = browser.html
soup = bs(html, 'html.parser')

In [13]:
featured_image_link = soup.find_all('img', class_='fancybox-image')[0]['src']

In [14]:
featured_image_url = f'{featured_img}{featured_image_link}'
featured_image_url

'https://spaceimages-mars.com/image/featured/mars1.jpg'

In [15]:
# browser.quit()

## Table

In [16]:
table = pd.read_html(table_url)

In [17]:
table_df = table[0].rename({0:'Description', 1:table[0][1][0], 2:table[0][2][0]}, axis='columns').set_index('Description')
table_df

,Mars,Earth
Description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [18]:
table_df.to_html('table.html')

## Hemisphere

In [19]:
browser.visit(hemispheres_url)

time.sleep(1)

In [20]:
hemisphere_image_urls = []
html = browser.html
soup = bs(html, 'html.parser')

for n in range(0, len(soup.find_all('div', class_='description'))):
    click_text = soup.find_all('div', class_='description')[n].h3.text
    browser.links.find_by_partial_text(click_text).click()
    
    time.sleep(1)
    
    html = browser.html
    soup = bs(html, 'html.parser')
    
    img_url = f'{hemispheres_url}{soup.find_all("a", text="Sample")[0]["href"]}'
    
    url_dict={}
    
    url_dict["title"] = click_text
    url_dict["img_url"] = img_url
    hemisphere_image_urls.append(url_dict)
    
    
    
    
    browser.links.find_by_partial_text('Back').click()
    
    time.sleep(1)
    
    html = browser.html
    soup = bs(html, 'html.parser')

browser.quit()

In [21]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]